In [ ]:
INIT=True
if INIT:
  !pip uninstall torch -y
  !pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cuXXX/torch_stable.html
  !pip install einops
  !pip install bitsandbytes
  !pip install sentencepiece
  !pip install accelerate
  !pip install transformers
  !pip install datasets
  !pip install optuna
import os

from google.colab import drive

Found existing installation: torch 2.0.1+cu118
Uninstalling torch-2.0.1+cu118:


In [ ]:
if INIT:
  drive.mount('/content/drive')
  ! pip install kaggle

  !mkdir ~/.kaggle


  ! cp kaggle.json ~/.kaggle/
  ! chmod 600 ~/.kaggle/kaggle.json

  ! kaggle competitions download kaggle-llm-science-exam
  ! unzip -o kaggle-llm-science-exam.zip -d   kaggle-llm-science-exam

  ! kaggle datasets download -d radek1/additional-train-data-for-llm-science-exam
  ! unzip -o  additional-train-data-for-llm-science-exam.zip -d    additional-train-data-for-llm-science-exam
  ! kaggle datasets download -d mpp1998/data-llm
  ! unzip -o  data-llm.zip -d    data-llm
  ! kaggle datasets download -d cdeotte/60k-data-with-context-v2
  ! unzip -o  60k-data-with-context-v2.zip -d    60k-data-with-context-v2
  ! kaggle datasets download -d mpp1998/my-small-dfs
  ! unzip -o  my-small-dfs.zip -d    my-small-dfs

In [ ]:
if False:
    !pip install datasets
    !pip install transformers
    !pip install transformers[torch]
    !pip install accelerate -U


In [ ]:
import sys
sys.version

In [ ]:

def map3(predictions, labels):
    hits = (-predictions).argsort() == labels.unsqueeze(1)
    return (hits[:, 0] * 1 + hits[:, 1] * 1/2 + hits[:, 2] * 1/3).sum().item() / hits.shape[0]

In [ ]:
# Importing Libraries

# Data Handling
import pandas as pd
import numpy as np
from dataclasses import dataclass
from typing import Optional, Union
from datasets import Dataset


import matplotlib.pyplot as plt


# Statistics & Mathematics
import scipy.stats as stats
from scipy.stats import shapiro, skew
import math

# RFECV for feature selection
from sklearn.feature_selection import RFECV

# Machine Learning Pipeline & process
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

# Preprocessing data
from sklearn.preprocessing import RobustScaler, StandardScaler, QuantileTransformer, FunctionTransformer
from sklearn.compose import ColumnTransformer

# Model Selection for Cross Validation
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

# Machine Learning metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, cohen_kappa_score, make_scorer

# ML regressors
from sklearn.linear_model import HuberRegressor,RANSACRegressor, TheilSenRegressor
from sklearn.ensemble import HistGradientBoostingRegressor, StackingRegressor, AdaBoostRegressor, RandomForestRegressor


# ML classifiers
from sklearn.ensemble import HistGradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.ensemble import StackingClassifier, VotingClassifier


# Clustering model
from sklearn.cluster import KMeans



# Randomizer
import random

# Encoder of categorical variables
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

# Importing HuggingFace's Transformers
from transformers import AutoTokenizer, AutoModelForMultipleChoice, Trainer, TrainingArguments, EarlyStoppingCallback
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy

# PyTorch
import torch

# Hiding warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Configuring Pandas to exhibit larger columns
'''
This is going to allow us to read the questions and answers
'''
pd.set_option('display.max_colwidth', 1000)

In [ ]:
# Checking if GPU is available
if torch.cuda.is_available():
    print("GPU is available")
    device = torch.device('cuda')
else:
    print("GPU is not available")
    device = torch.device('cpu')

In [ ]:
# Defining seed and the template for plots
seed = 42
plotly_template = 'simple_white'

In [ ]:
df = pd.read_csv('./60k-data-with-context-v2/train_with_context2.csv') # Loading data
df.shape


> <p style="font-size: 20px"><b>📝 We have a total of 200 questions. <br><br>
    📝 All features, except for <code>id</code>, are categorical features.</b></p>

In [ ]:
if "id" in df.columns:
  df = df.drop('id', axis = 1)  # Dropping 'Id' columns

In [ ]:
# Concatenating original dataframe to extra dataframes
augmented_df =  pd.read_csv('/content/my-small-dfs/adjusted_distribution_train_20_000samples.csv')



augmented_df=augmented_df.loc[~augmented_df.prompt.isin(df.prompt)].dropna()
augmented_df=augmented_df.drop_duplicates(subset="prompt")
augmented_df.shape


In [ ]:
augmented_df = augmented_df[[ 'prompt', 'context', 'A', 'B', 'C', 'D', 'E', 'answer',]]
augmented_df.head(3)

> <p style="font-size: 20px"><b>📝 We went from 200 samples to 700 samples.</b></p>

<p style="font-size: 20px">Now I'm going to split the data, creating a dataframe for training and another for validation. I am going to use 70% of the data for training and 30% for validation.</p>

In [ ]:
# Creating training and validation sets
train_df =augmented_df
val_df = df

In [ ]:
train_df.shape

In [ ]:
train_df .isna() .sum(  )

In [ ]:
folding =False

from sklearn.model_selection import KFold,StratifiedKFold
n_splits = 5
if folding:

    # Crear una instancia de KFold
    kfold = StratifiedKFold(n_splits=n_splits,random_state=21,shuffle=True)

    for fold,(train_indice, test_indice) in enumerate(kfold.split(    augmented_df,    augmented_df.answer)):
      if fold ==3:
        print(fold)

        train_df=augmented_df.iloc[train_indice]
print(train_df.shape)
train_df=train_df.fillna("MASK_NAS")
train_df=train_df.loc[~train_df.prompt.isin(df.prompt)]

print(train_df.loc[train_df.prompt.isin(df.prompt)].shape)
train_df =train_df .drop_duplicates(subset="prompt")
print(train_df.shape)

In [ ]:
# Converting dataframes into datasets
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)

print('\nTrain Dataset:\n')
print(train_ds)
print('\nValidation Dataset:\n')
print(val_ds)

<p style="font-size: 20px">Now we convert the dataframes to Datasets.</p>

<p style="font-size: 20px">We are now going to use the <code>AutoTokenizer</code> class from Hugging Face, as well as the <code>from_pretrained()</code> method, to load the token vocabulary pretrained for the de DeBERTa model.</p>

In [ ]:
def competition_score(y_true, y_pred):

    """
    Obtaining score for the model
    """

    ap_at_3 = 0.0
    for i in range(len(y_true)):
        if y_true[i]==  y_pred[i][0]:
            ap_at_3 += 1
        elif y_true[i]==  y_pred[i][1]:
            ap_at_3 += 1/2
        elif y_true[i]==  y_pred[i][2]:
            ap_at_3 += 1/3


    map3 = ap_at_3 / len(y_true)
    return map3

def predictions_to_map_output(predictions):
    sorted_answer_indices = np.argsort(-predictions) # Sortting indices in descending order
    top_answer_indices = sorted_answer_indices[:,:3] # Taking the first three indices for each row
    top_answers = np.vectorize(index_to_option.get)(top_answer_indices) # Transforming indices to options - i.e., 0 --> A
    return  top_answers
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    y_pred = predictions_to_map_output(logits)
    y_true = [index_to_option[label] for label in labels]
    print( y_true)
    return {'Map@3': np.round(competition_score(y_true, y_pred), 3)}

In [ ]:
train_df["all"]=train_df["prompt"]+train_df["A"]+train_df["B"]+train_df["C"]+train_df["D"]+train_df["E"]

In [ ]:
# Setting up answer choices and their respective indices
options = 'ABCDE'
indices = list(range(5)) # Indexing 0, 1, 2, 3, 4 for each option


option_to_index = {option: index for option, index in zip(options, indices)} # Converting options to indices '''A to 0'''
index_to_option = {index: option for option, index in zip(options, indices)} # Converting indices to options '''0 to A'''

def preprocess_wrapper(tokenizer,MAX_INPUT=350):
    def preprocess(example):

        first_sentence = [example['prompt']] * 5 # Repeating the same question 5 times
        second_sentence = [] # Creating list of possible answers
        for option in options:
            second_sentence.append(example[option])

        # The tokenizer converts the question and answers into 'tokens'.
        # 'tokens' are simply a sequence of integers in which each specific integer corresponds to
        # a word or subword that BERT is capable of comprehending
        tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)

        # Indexing label - A,B,C,D or E - to either 0, 1, 2, 3, 4, or 5
        tokenized_example['label'] = option_to_index[example['answer']]

        # tokenized_example returns:
            # input_ids --> List of lists represents the tokens
            # token_type_ids --> A list of lists indicating whether each token belongs to the 1st or 2nd sentence
            # attention_mask --> List of list where each inner list is either 1 or 0. 1 are not padding tokens, 0 are padding tokens
            # label --> The index for the correct option
        return tokenized_example

    def preprocess_with_context(example):

      first_sentence = [ "[CLS] " + example['context'] ] * 5
      second_sentences = [" #### " + example['prompt'] + " [SEP] " + example[option] + " [SEP]" for option in 'ABCDE']

      #tokenized_example = tokenizer(first_sentence, second_sentences, truncation=True)
      tokenized_example = tokenizer(first_sentence, second_sentences, truncation='only_first',
                                    max_length=MAX_INPUT, add_special_tokens=False)
      tokenized_example['label'] = option_to_index[example['answer']]

      return tokenized_example

    return preprocess_with_context

In [ ]:
@dataclass
class DataCollatorForMultipleChoice:
    '''
    This class is designed to handle the formatting and batching the data for mutiple-choice tasks
    '''

    # The tokenizer to be used for tokenizing the data
    tokenizer: PreTrainedTokenizerBase

    # The strategy to be used for padding the data
    padding: Union[bool, str, PaddingStrategy] = True

    # The maximum length for any input sequence
    max_length: Optional[int] = None

    # If provided, pad the sequences to a multiple of this value
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        # Finding the correct label key in the features
        label_name = "label" if 'label' in features[0].keys() else 'labels'

        # Extracting the labels and removing them from features
        labels = [feature.pop(label_name) for feature in features]

        # Obtaining batch size
        batch_size = len(features)

        # Obtaining number of choices
        num_choices = len(features[0]['input_ids'])

        # Reestructuring features so each question-choice pair becomes a separate example
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        # Padding all sequences to the same length
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )

        # Reshaping the batch back into the original format
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}

        # Adding the labels back into the batch as a tensor
        batch['labels'] = torch.tensor(labels,
                                       dtype=torch.int64)

        # Returning the batch
        return batch

In [ ]:
model_name="/content/drive/MyDrive/checkpoit30456/checkpoints/checkpoint-30456"



In [ ]:

print(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Instantiating model
model = AutoModelForMultipleChoice.from_pretrained(model_name)
model = model.to(device) # GPU





In [ ]:
tokenizer("a # b")

In [ ]:
for param in model.parameters():
  if not param.requires_grad:
    print(param.requires_grad )
  param.requires_grad =True


In [ ]:
len([ layer for layer in model.deberta.encoder.layer])

In [ ]:
FREEZE_EMBEDDINGS = True
FREEZE_LAYERS=18
if FREEZE_EMBEDDINGS:
    print('Freezing embeddings.')
    for param in model.deberta.embeddings.parameters():
        param.requires_grad = False
if FREEZE_LAYERS>0:
    print(f'Freezing {FREEZE_LAYERS} layers.')
    for layer in model.deberta.encoder.layer[:FREEZE_LAYERS]:
        for param in layer.parameters():
            param.requires_grad = False

In [ ]:
model_dir = 'finetuned_deberta' # Directory to save model and files
output_dir=model_dir
checkpoint_dir="checkpoints"
# Tokenizing train Dataset
r_cols= [e for e in ['prompt', 'A', 'B', 'C', 'D', 'E', 'answer', '__index_level_0__','context','source'] if e in train_ds.features]
tokenized_train_ds = train_ds.map(preprocess_wrapper(tokenizer,MAX_INPUT=450), batched=False, remove_columns=r_cols)


In [ ]:

r_cols= [e for e in ['prompt', 'A', 'B', 'C', 'D', 'E', 'answer', '__index_level_0__','context','source']  if e in val_ds.features]
tokenized_val_ds = val_ds.map(preprocess_wrapper(tokenizer,MAX_INPUT=650), batched=False,
                              remove_columns=r_cols)

# Defining parameters
training_args = TrainingArguments(
    output_dir=checkpoint_dir,
    warmup_ratio = .8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=5,
    learning_rate= 5e-6,
    per_device_train_batch_size= 4,
    per_device_eval_batch_size= 4,
    #gradient_accumulation_steps = 2,
    gradient_checkpointing = True,
    num_train_epochs=20,
    #weight_decay=0.01,
    metric_for_best_model = 'eval_loss',
    report_to='none',
)
# Defining Trainer to train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_val_ds,

    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics = compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 5,
                                      early_stopping_threshold = 0.01)]
)


print(trainer.evaluate())
trainer.train()

In [ ]:
model.save_pretrained(model_dir)
# Save the tokenizer as well
tokenizer.save_pretrained(model_dir)
print(trainer.evaluate())

In [ ]:
# Source directory to be zipped
import shutil
source_dir = f"drive/MyDrive/{output_dir}"




# Create a zip archive of the directory
shutil.make_archive(source_dir , 'zip', output_dir)

print(f"Zip archive '{source_dir}' created.")

In [ ]:
# Source directory to be zipped
import shutil

for dir in os.listdir("checkpoints"):
  try:
    print(dir)

    source_dir = f"drive/MyDrive/checkpoints/{dir}"




    # Create a zip archive of the directory
    shutil.make_archive(source_dir , 'zip', f"checkpoints/{dir}")

    print(f"Checkpoints Zip archive '{source_dir}' created.")
  except Exception as e:
    print(e)